# Analysis of experiments
### Gradual introduction of True Price products
Based on Sub-model 2.2 with added mechanism for the gradual introduction of TP products

In [18]:
import numpy as np
import pandas as pd
import seaborn as sns
import networkx as nx
import cProfile
import pstats
import mesa
import joblib
import matplotlib.pyplot as plt
from collections import Counter
from tqdm import tqdm
from SALib.sample import saltelli
from SALib.analyze import sobol
from SALib.test_functions import Ishigami
import matplotlib.ticker as mtick
from joblib import Parallel, delayed
import json


In [17]:
from experiments_model_copy import ConsumatModel

In [19]:
base_config = {
    'TP_percentage': 0.4,
    'tp_introduction_rate': 0.1,
    'satisfaction_threshold': 0.5,
    'uncertainty_threshold': 0.5,
    'product_price_range': (5, 10),
    'min_increase_percentage': 4,
    'max_increase_percentage': 10,
    'num_products': 20,
    'inflation_rate': 3,
    'epsilon': 0.5,   
    'comparison_attributes': ['budget', 'preference_conformity', 'preference_sustainability'],
    'seed': 42  
}

watts_strogatz_config = base_config.copy()
watts_strogatz_config.update({
    'network_type': 'watts_strogatz',
    'network_params': {
        'n': 1600,
        'k': 8,
        'p': 0.3
    }
})


barabasi_albert_config = base_config.copy()
barabasi_albert_config.update({
    'network_type': 'barabasi_albert',
    'network_params': {
        'n': 1600,
        'm': 8
    }
})

random_regular_config = base_config.copy()
random_regular_config.update({
    'network_type': 'random_regular',
    'network_params': {
        'n': 1600,
        'd': 8
    }
})

holme_kim_config = base_config.copy()
holme_kim_config.update({
    'network_type': 'holme_kim',
    'network_params': {
        'n': 1600,
        'm': 8,
        'p': 0.3
    }
})
network_configs = {
    'Watts-Strogatz': watts_strogatz_config,
    'Barabasi-Albert': barabasi_albert_config,
    'Random Regular': random_regular_config,
    'Holme-Kim': holme_kim_config
}

tp_introduction_rates = np.arange(0, 1.1, 0.1)

num_steps = 10
all_results = {network: {} for network in network_configs}
final_adoption_rates = {network: [] for network in network_configs}

def collect_simulation_data(model, num_steps):
    collected_data = {
        'Step': [],
        'Adoption Rate': []
    }

    for step in tqdm(range(num_steps), desc='Simulation Progress'):
        model.step()
        collected_data['Step'].append(step)
        collected_data['Adoption Rate'].append(model.calculate_true_price_adoption_rate())

    model_data = pd.DataFrame(collected_data)
    return model_data

def plot_metric(data, metric, ylabel, filename, scale_as_percentage=False):
    plt.figure(figsize=(15, 8))
    colors = plt.cm.viridis(np.linspace(0, 1, len(data)))

    for color, (tp, df) in zip(colors, data.items()):
        tp_percentage_label = f'{tp:.1f}'  
        if scale_as_percentage:
            plt.plot(df['Step'], df[metric], label=tp_percentage_label, color=color, marker='o')
        else:
            plt.plot(df['Step'], df[metric], label=tp_percentage_label, color=color, marker='o')

    plt.xlabel('Step', fontsize=14)
    plt.ylabel(ylabel, fontsize=14)
    plt.legend(title='TP Rate', fontsize=12)
    plt.grid(True)

    if scale_as_percentage:
        plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter())

    
    plt.savefig(f'experiment_rate_{filename}.png')
    plt.close()

for network_type, config in network_configs.items():
    for tp in tp_introduction_rates:
        config['tp_introduction_rate'] = tp
        model = ConsumatModel(config)
        model_data = collect_simulation_data(model, num_steps)
        all_results[network_type][tp] = model_data
        final_adoption_rate = model_data['Adoption Rate'].iloc[-1]
        final_adoption_rates[network_type].append(final_adoption_rate)

metrics = [
    ('Adoption Rate', 'Adoption Rate (%)')
]

for network_type, data in all_results.items():
    for metric, ylabel in metrics:
        plot_metric(data, metric, ylabel, f'{network_type.lower().replace(" ", "_")}_{metric.lower().replace(" ", "_")}', False)

for network_type, rates in final_adoption_rates.items():
    plt.figure(figsize=(12, 10))
    plt.plot(tp_introduction_rates, rates, marker='o')
    plt.xlabel('TP introduction rate ', fontsize=14)
    plt.ylabel('Final True Price Adoption Rate (%)', fontsize=14)
    plt.ylim(0, 100)
    plt.grid(True)
    plt.savefig(f'experiment_final_true_price_adoption_rate_{network_type.lower().replace(" ", "_")}.png')
    plt.close()












Simulation Progress: 100%|██████████| 10/10 [00:32<00:00,  3.23s/it]











Simulation Progress: 100%|██████████| 10/10 [00:32<00:00,  3.25s/it]











Simulation Progress: 100%|██████████| 10/10 [00:38<00:00,  3.88s/it]


In [20]:
base_config = {
    'TP_percentage': 0.4,
    'tp_introduction_rate': 0.3,
    'satisfaction_threshold': 0.5,
    'uncertainty_threshold': 0.5,
    'product_price_range': (5, 10),
    'min_increase_percentage': 4,
    'max_increase_percentage': 10,
    'num_products': 20,
    'inflation_rate': 3,
    'epsilon': 0.5,   
    'comparison_attributes': ['budget', 'preference_conformity', 'preference_sustainability'],
    'seed': 42  
}

watts_strogatz_config = base_config.copy()
watts_strogatz_config.update({
    'network_type': 'watts_strogatz',
    'network_params': {
        'n': 1600,
        'k': 8,
        'p': 0.3
    }
})


barabasi_albert_config = base_config.copy()
barabasi_albert_config.update({
    'network_type': 'barabasi_albert',
    'network_params': {
        'n': 1600,
        'm': 8
    }
})

random_regular_config = base_config.copy()
random_regular_config.update({
    'network_type': 'random_regular',
    'network_params': {
        'n': 1600,
        'd': 8
    }
})

holme_kim_config = base_config.copy()
holme_kim_config.update({
    'network_type': 'holme_kim',
    'network_params': {
        'n': 1600,
        'm': 8,
        'p': 0.3
    }
})
network_configs = {
    'Watts-Strogatz': watts_strogatz_config,
    'Barabasi-Albert': barabasi_albert_config,
    'Random Regular': random_regular_config,
    'Holme-Kim': holme_kim_config
}

tp_percentages = np.arange(0, 1.05, 0.05)
num_steps = 10
all_results = {network: {} for network in network_configs}
final_adoption_rates = {network: [] for network in network_configs}

def collect_simulation_data(model, num_steps):
    collected_data = {
        'Step': [],
        'Adoption Rate': []
    }

    for step in tqdm(range(num_steps), desc='Simulation Progress'):
        model.step()
        collected_data['Step'].append(step)
        collected_data['Adoption Rate'].append(model.calculate_true_price_adoption_rate())

    model_data = pd.DataFrame(collected_data)
    return model_data

def plot_metric(data, metric, ylabel, filename, scale_as_percentage=False):
    plt.figure(figsize=(15, 8))
    colors = plt.cm.viridis(np.linspace(0, 1, len(data)))

    for color, (tp, df) in zip(colors, data.items()):
        tp_percentage_label = f'{tp * 100:.1f}%'  # Convert TP to percentage for the legend
        if scale_as_percentage:
            plt.plot(df['Step'], df[metric], label=tp_percentage_label, color=color, marker='o')
        else:
            plt.plot(df['Step'], df[metric], label=tp_percentage_label, color=color, marker='o')

    plt.xlabel('Step', fontsize=14)
    plt.ylabel(ylabel, fontsize=14)
    plt.legend(title='TP Percentage', fontsize=12)
    plt.grid(True)

    if scale_as_percentage:
        plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter())

    
    plt.savefig(f'experiment_TP_{filename}.png')
    plt.close()

for network_type, config in network_configs.items():
    for tp in tp_percentages:
        config['TP_percentage'] = tp
        model = ConsumatModel(config)
        model_data = collect_simulation_data(model, num_steps)
        all_results[network_type][tp] = model_data
        final_adoption_rate = model_data['Adoption Rate'].iloc[-1]
        final_adoption_rates[network_type].append(final_adoption_rate)

metrics = [
    ('Adoption Rate', 'Adoption Rate (%)')
]

for network_type, data in all_results.items():
    for metric, ylabel in metrics:
        plot_metric(data, metric, ylabel, f'{network_type.lower().replace(" ", "_")}_{metric.lower().replace(" ", "_")}', False)

for network_type, rates in final_adoption_rates.items():
    plt.figure(figsize=(12, 10))
    plt.plot(tp_percentages*100, rates, marker='o')
    plt.xlabel('TP introduction percentage (%)', fontsize=14)
    plt.ylabel('Final True Price Adoption Rate (%)', fontsize=14)
    plt.ylim(0, 100)
    plt.grid(True)
    plt.savefig(f'experiment_TP_final_true_price_adoption_rate_{network_type.lower().replace(" ", "_")}.png')
    plt.close()

Simulation Progress: 100%|██████████| 10/10 [00:18<00:00,  1.89s/it]


### Sensitivity Analysis

In [12]:


base_config = {
    'TP_percentage': 0.7,
    'tp_introduction_rate': 0.1,
    'satisfaction_threshold': 0.5,
    'uncertainty_threshold': 0.5,
    'product_price_range': (5, 10),
    'min_increase_percentage': 4,
    'max_increase_percentage': 10,
    'num_products': 20,
    'inflation_rate': 3,
    'epsilon': 0.5,
    'comparison_attributes': ['budget', 'preference_sustainability', 'preference_conformity'],
    'seed': 42  
}

# Configuration for Watts-Strogatz network
watts_strogatz_config = base_config.copy()
watts_strogatz_config.update({
    'network_type': 'watts_strogatz',
    'network_params': {
        'n': 1600,
        'k': 8,
        'p': 0.3
    }
})


# Configuration for Barabasi-Albert network
barabasi_albert_config = base_config.copy()
barabasi_albert_config.update({
    'network_type': 'barabasi_albert',
    'network_params': {
        'n': 1600,
        'm': 8
    }
})

# Configuration for Random Regular network
random_regular_config = base_config.copy()
random_regular_config.update({
    'network_type': 'random_regular',
    'network_params': {
        'n': 1600,
        'd': 8
    }
})

# Configuration for Holme-Kim network
holme_kim_config = base_config.copy()
holme_kim_config.update({
    'network_type': 'holme_kim',
    'network_params': {
        'n': 1600,
        'm': 8,
        'p': 0.3
    }
})


network_configs = {
    'Watts-Strogatz': watts_strogatz_config,
    'Barabasi-Albert': barabasi_albert_config,
    'Random Regular': random_regular_config,
    'Holme-Kim': holme_kim_config
}

network_configurations = {
    'Watts-Strogatz': watts_strogatz_config,
    'Barabasi-Albert': barabasi_albert_config,
    'Random Regular': random_regular_config,
    'Holme-Kim': holme_kim_config
}

network_problems = {
    'Watts-Strogatz': {
        'num_vars': 9,
        'names': ['TP_percentage', 'satisfaction_threshold', 'uncertainty_threshold', 'epsilon', 'min_increase_percentage','max_increase_percentage', 'k', 'p', 'tp_introduction_rate'],
        'bounds': [[0, 1], [0, 1], [0, 1], [0,1], [1,10], [11,20], [2, 10], [0, 1], [0,1]] 
    },
    'Barabasi-Albert': {
        'num_vars': 8,
        'names': ['TP_percentage', 'satisfaction_threshold', 'uncertainty_threshold','epsilon','min_increase_percentage', 'max_increase_percentage', 'm', 'tp_introduction_rate'],
        'bounds': [[0, 1], [0, 1], [0, 1], [0,1],[1,10], [11,20],[2, 10], [0,1]]  
    },
    'Random Regular': {
        'num_vars': 8,
        'names': ['TP_percentage', 'satisfaction_threshold', 'uncertainty_threshold', 'epsilon','min_increase_percentage','max_increase_percentage','d', 'tp_introduction_rate'],
        'bounds': [[0, 1], [0, 1], [0, 1], [0,1],[1,10], [11,20],[2, 10], [0,1]]  
    },
    'Holme-Kim': {
        'num_vars': 9,
        'names': ['TP_percentage', 'satisfaction_threshold', 'uncertainty_threshold', 'epsilon','min_increase_percentage','max_increase_percentage','m', 'p', 'tp_introduction_rate'],
        'bounds': [[0, 1], [0, 1], [0, 1],[0,1], [1,10], [11,20],[2, 10], [0, 1], [0,1]] 
    }
}

In [13]:
num_samples = 256

samples = {}
for network_name, problem in network_problems.items():
    samples[network_name] = saltelli.sample(problem, num_samples)


/var/folders/53/cl5y6r2x7n5_942j6h59yyh00000gn/T/ipykernel_6592/3049819496.py:7: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5. Please use `salib.sample.sobol`
  samples[network_name] = saltelli.sample(problem, num_samples)


In [14]:


def collect_simulation_data(model, num_steps):
    collected_data = {
        'Step': [],
        'Adoption Rate': [],
        'Avg_F_Satisfaction': [],
        'Avg_S_Satisfaction': [],
        'Avg_P_Satisfaction': [],
        'Avg_F_Uncertainty': [],
        'Avg_S_Uncertainty': [],
        'Avg_P_Uncertainty': []
    }

    for step in range(num_steps):
        model.step()
        collected_data['Step'].append(step)
        collected_data['Adoption Rate'].append(model.calculate_true_price_adoption_rate())
        collected_data['Avg_F_Satisfaction'].append(np.mean([agent.F_satisfaction for agent in model.schedule.agents]))
        collected_data['Avg_S_Satisfaction'].append(np.mean([agent.S_satisfaction for agent in model.schedule.agents]))
        collected_data['Avg_P_Satisfaction'].append(np.mean([agent.P_satisfaction for agent in model.schedule.agents]))
        collected_data['Avg_F_Uncertainty'].append(np.mean([agent.F_uncertainty for agent in model.schedule.agents]))
        collected_data['Avg_S_Uncertainty'].append(np.mean([agent.S_uncertainty for agent in model.schedule.agents]))
        collected_data['Avg_P_Uncertainty'].append(np.mean([agent.P_uncertainty for agent in model.schedule.agents]))

    model_data = pd.DataFrame(collected_data)
    return model_data

def run_single_simulation(params, config, network_name, num_steps):
    config = config.copy()
    config['TP_percentage'] = params[0]
    config['satisfaction_threshold'] = params[1]
    config['uncertainty_threshold'] = params[2]
    config['epsilon'] = params[3]
    config['min_increase_percentage'] = params[4]
    config['max_increase_percentage'] = params[5]
    config['tp_introduction_rate'] = params[-1]

    if network_name == 'Watts-Strogatz':
        config['network_params']['k'] = int(params[6])
        config['network_params']['p'] = params[7]
    elif network_name == 'Barabasi-Albert':
        config['network_params']['m'] = int(params[6])
    elif network_name == 'Random Regular':
        config['network_params']['d'] = int(params[6])
    elif network_name == 'Holme-Kim':
        config['network_params']['m'] = int(params[6])
        config['network_params']['p'] = params[7]

    model = ConsumatModel(config)
    model_data = collect_simulation_data(model, num_steps)
    final_step_data = model_data.iloc[-1]

    return [
        final_step_data['Adoption Rate'],
        final_step_data['Avg_F_Satisfaction'],
        final_step_data['Avg_S_Satisfaction'],
        final_step_data['Avg_P_Satisfaction'],
        final_step_data['Avg_F_Uncertainty'],
        final_step_data['Avg_S_Uncertainty'],
        final_step_data['Avg_P_Uncertainty']
    ]

def run_gsa_simulation_parallel(network_name, config, samples, num_steps, n_jobs=-1):
    all_results = Parallel(n_jobs=n_jobs)(
        delayed(run_single_simulation)(params, config, network_name, num_steps)
        for params in tqdm(samples, desc=f'Running simulations for {network_name}')
    )
    return np.array(all_results)

num_steps = 10

for network_name, config in network_configurations.items():
    print(f"Running GSA for {network_name} network...")
    config = base_config.copy()
    config.update(network_configs[network_name])
    all_results = run_gsa_simulation_parallel(network_name, config, samples[network_name], num_steps)
    
    results = {}
    
    for i, output in enumerate(['Adoption Rate', 'Avg_F_Satisfaction', 'Avg_S_Satisfaction', 'Avg_P_Satisfaction', 'Avg_F_Uncertainty', 'Avg_S_Uncertainty', 'Avg_P_Uncertainty']):
        Si = sobol.analyze(network_problems[network_name], all_results[:, i], print_to_console=True)
        results[output] = {
            'S1': Si['S1'].tolist(),
            'ST': Si['ST'].tolist(),
            'S2': Si['S2'].tolist()
        }
        print(f'Sensitivity analysis for {output} ({network_name} network):')
        print(Si)
    
    with open(f'experiments_sensitivity_results_{network_name}.json', 'w') as f:
        json.dump(results, f)


Running GSA for Holme-Kim network...


KeyboardInterrupt: 